#### Part I - Detecting the object boundary
Detection and localization of a prominent straight edge in live video
- Capture an image from a video camera.
- Use an edge detector (such as Canny) to create an edge image.
- Use the Hough Transform (in OpenCV) to locate four prominent lines in the image.
- Display the lines in the live image.
- Adjust the parameters of the edge detector and the Hough Transform for best results at video rate.

#### Part II - Rectification
- Compute and enumerate the intersections of the lines defining the four corners of the quadrangle.
- Using the four corner locations, create a perspective transformation that maps to the corners of a new image, and warp the image content to the new image.
- Display the rectified image.

#### Test:
1. How well your straight line detector follows the edge of a sheet of paper moved across the field of view of the camera? It accurately follows the paper's edges, even when tilted or moving, as long as edges are clearly visible.
2. How well it detects other straight lines in your environment? It performs well but I tuned Canny's parameters to to prioritize significant edges, reducing sensitivity to finer details.
3. The processing time for one video frame or image? Processes frames at ~30 FPS.

In [20]:
import math
import numpy as np
from scipy.spatial import ConvexHull

def line_to_homogeneous(rho, theta):
    """
    Convert a line in polar coordinates to homogeneous form: ax + by + c = 0
    """
    a = math.cos(theta)
    b = math.sin(theta)
    c = -rho
    return np.array([a, b, c])

def cross_product(line1, line2):
    """
    Compute the intersection of two lines using the cross product in homogeneous coordinates.
    Returns (x, y) in pixel coordinates or None if the lines are parallel.
    """
    intersection = np.cross(line1, line2)
    if abs(intersection[2]) < 1e-10:  # Check if the last coordinate is close to zero (parallel lines)
        return None
    x, y = intersection[0] / intersection[2], intersection[1] / intersection[2]
    return int(x), int(y)

def order_points(pts):
    """
    Order points in the following order: top-left, top-right, bottom-right, bottom-left.
    """
    rect = np.zeros((4, 2), dtype="float32")
    s = pts.sum(axis=1)
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]

    diff = np.diff(pts, axis=1)
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]

    return rect

def get_convex_hull(points):
    """Find the four corners using a convex hull."""
    points = np.array(points, dtype=np.float32)  # Ensure it's a 2D array
    if len(points) < 4:
        return None  # Not enough points to find corners
    hull = ConvexHull(points)
    hull_points = points[hull.vertices]
    if len(hull_points) < 4:
        return None  # Convex hull must have at least 4 points
    return hull_points[:4]  # Return the first 4 hull points

import itertools

def get_four_corners(points):
    """Find four points forming the largest quadrilateral."""
    if len(points) < 4:
        return None
    max_area = 0
    best_quad = None
    for quad in itertools.combinations(points, 4):
        rect = np.array(quad, dtype="float32")
        area = cv2.contourArea(rect)
        if area > max_area:
            max_area = area
            best_quad = rect
    return best_quad


In [23]:
import cv2
import time
import numpy as np

cap = cv2.VideoCapture(0)

prev_time = 0.00
while(True):
    current_time = time.time()
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Edeg detection
    edges = cv2.Canny(gray, 
                      100, 
                      200, 
                      apertureSize=3, # modify the size of the Sobel kernel from 5 to 3
                      L2gradient=True) # modify the gradient calculation from L1 to L2
    cv2.imshow('edges',edges)
    
    # Hough Line Transform to detect lines
    lines = cv2.HoughLines(edges, 
                           1, # tried increasing rho for lower precision and fater processing (2 and 5) but worse results
                           np.pi / 180, # decrease theta for higher angular precision (360)
                           100, # increase threshold to avoid detecting noises (from 100 to 150) 
                           None, 
                           0, 
                           0)

    intersections = []
    homogeneous_lines = []
    if lines is not None:
        # Convert all lines to homogeneous form
        for line in lines:
            rho, theta = line[0]
            homogeneous_lines.append(line_to_homogeneous(rho, theta))

        # Draw lines and compute intersections
        for i, line1 in enumerate(homogeneous_lines):
            for j, line2 in enumerate(homogeneous_lines[i + 1:]):
                intersect = cross_product(line1, line2)
                if intersect is not None:
                    intersections.append(intersect)

        # Draw the detected lines
        for rho, theta in lines[:, 0]:
            a = math.cos(theta)
            b = math.sin(theta)
            x0 = a * rho
            y0 = b * rho
            pt1 = (int(x0 + 1000 * (-b)), int(y0 + 1000 * a))
            pt2 = (int(x0 - 1000 * (-b)), int(y0 - 1000 * a))
            cv2.line(frame, pt1, pt2, (0, 0, 255), 1, cv2.LINE_AA)

        # Draw the intersections
        # for point in intersections:
        #     cv2.circle(frame, point, 5, (0, 255, 0), -1)  # Mark intersections

        # Select four corners and order them
        
        if len(intersections) >= 4:
            corners = np.array(intersections[:4], dtype="float32")  # Naive selection of the first 4
            ordered_corners = order_points(corners)

        # intersections = np.array(intersections, dtype=np.float32)
        # hull_points = get_convex_hull(intersections)
        # if hull_points is not None:
        #     ordered_corners = order_points(hull_points)

        # corners = get_four_corners(intersections)
        # if corners is not None:
        #     ordered_corners = order_points(corners)

            for i, point in enumerate(ordered_corners):
                x, y = int(point[0]), int(point[1])
                cv2.circle(frame, (x, y), 5, (255,0,0), -1) 

            # Define the target points for the rectified image
            width = 640  # Desired width of the rectified image
            height = 640  # Desired height of the rectified imageq
            dst = np.array([
                [0, 0],
                [width - 1, 0],
                [width - 1, height - 1],
                [0, height - 1]
            ], dtype="float32")

            # Compute the perspective transform matrix
            M = cv2.getPerspectiveTransform(ordered_corners, dst)
     
            # Warp the image to get a rectified top-down view
            rectified = cv2.warpPerspective(frame, M, (width, height))

            # Display the rectified image
            if rectified is not None and rectified.size > 0:
                cv2.imshow("Rectified Image", rectified)
            else:
                print("Error: Rectified image is empty.") #############################################
            
    # Calculate FPS and display it
    fps = 1 / (current_time - prev_time) if prev_time else 0
    prev_time = current_time
    cv2.putText(
        frame,
        f"FPS: {fps:.2f}",
        (10, 30),
        cv2.FONT_HERSHEY_SIMPLEX,
        1,
        (255, 255, 255), 
        2,
        cv2.LINE_AA,
    )

    cv2.imshow('frame',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()